In [6]:
# Create docker container for agent environment
import docker
import os 
import dotenv

dotenv.load_dotenv()

# Create a Docker client from the environment
client = docker.from_env()

# Get GH token for authorization
gh_token = os.getenv("GH_TOKEN")
if not gh_token:
    raise Exception("GH_TOKEN is not set")

# Pull the image (Ubuntu and Git pre-installed)
# Use `ubuntu` base and install git during container creation
container = client.containers.run(
    image="ubuntu:latest",              # Base image
    command="sleep infinity",           # Keep container running
    detach=True,
    tty=True,                           # Enables command execution like a shell
    stdin_open=True
)

# Install git inside the container
exit_code, output = container.exec_run("bash -c 'apt-get update && apt-get install -y git'")
if exit_code != 0:
    _, stderr = output
    container.stop()
    container.remove()
    raise Exception(f"Failed to install git: {stderr.decode()}")

print(f"Successfully installed git on container: {container.id}")

# Configure git
# Write credentials
exit_code, output = container.exec_run("bash -c 'echo \"https://${GITHUB_TOKEN}@github.com\" > ~/.git-credentials'", environment={"GITHUB_TOKEN": gh_token})
if exit_code != 0:
    _, stderr = output
    container.stop()
    container.remove()
    raise Exception(f"Failed to write credentials: {stderr.decode()}")

# Store credentials
exit_code, output = container.exec_run("git config --global credential.helper store")
if exit_code != 0:
    _, stderr = output
    container.stop()
    container.remove()
    raise Exception(f"Failed to tell Git to use stored credentials: {stderr.decode()}")

print(f"Successfully configured git on container: {container.id}")


Successfully configured git on container: 0cb734bbc4586b342f2f34432c9e7518e977fc0fd7f327695aad959a2ec7affb


In [1]:
# Define tools

BASH_CMD = "bash -c"
GIT_CMD = "git"

def parse_command(cmd: str) -> bool:
    allowed_unix = ["ls", "cd", "pwd", "cat", "mkdir"]
    allowed_git = ["add", "commit", "push", "pull", "status"]
    allowed_commands = allowed_unix + allowed_git
    
    if cmd.split()[0] not in allowed_commands:
        return False
    
    return True

def execute_command(cmd: str) -> str:
    exit_code, output = container.exec_run(cmd)
    if exit_code != 0:
        _, stderr = output
        return f"Error: {stderr.decode()}"
    return output.decode()


def execute_unix_command(cmd: str) -> str:
    return execute_command(f"{BASH_CMD} {cmd}")

def execute_git_command(cmd: str) -> str:
    return execute_command(f"{BASH_CMD} {GIT_CMD} {cmd}")

In [ ]:
# Create docker container for agent environment

